In [2]:
import os
import tweepy
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [24]:
# Load environment variables from .env file
load_dotenv()

True

In [25]:
consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]

In [26]:
auth = tweepy.OAuth1UserHandler(
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

In [27]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [28]:
me = api.verify_credentials()

In [85]:
QUERY = "#COVID19 OR #CoronaVirus OR #Pandemic until:2022-06-25 -filter:retweets"

In [56]:
# basic keyword search with 100 tweets
# tweets = api.search_tweets(
#     QUERY, lang="en", count=10000, tweet_mode="extended")

In [79]:
import datetime
# specify the start and end dates for the range
start_date = datetime.datetime(2020, 1, 1, 0, 0, 0)
end_date = datetime.datetime(2022, 6, 24, 0, 0, 0)

In [86]:
all_tweets = []
for tweet in tweepy.Cursor(api.search_tweets, QUERY, count=1000, lang="en").items():
    all_tweets.append(tweet)

In [33]:
tweets =  tweepy.Cursor(api.search_tweets, q=QUERY).items(50000)

In [45]:
len(tweets)

0

In [87]:
len(all_tweets)

0

In [44]:
for i in tweets:
    print(i)